In [42]:
# %pip install tensorflow
# %pip install keras
# %pip install Keras-Preprocessing
# %pip install opencv-python

In [43]:
# import tensorflow as tf
# print("Versão do TensorFlow:", tf.__version__)
# import keras as K
# print("Versão do Keras:", K.__version__)


In [44]:
# Imports
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [45]:
# Inicializando a Rede Neural Convolucional
def build_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation=None, input_shape=input_shape),
        MaxPooling2D((2, 2)),
        #Removidas no teste 3
        Conv2D(64, (3, 3), activation=None),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=None),
        MaxPooling2D((2, 2)),
        #Camadas adicionadas no teste 2
        # Dense(128, activation='relu'),
        # Dense(num_classes, activation='softmax'),
        # Conv2D(64, (3, 3), activation='relu'),
        # MaxPooling2D((2, 2)),
        # Conv2D(64, (3, 3), activation='relu'),
        # MaxPooling2D((2, 2)),
        # Conv2D(128, (3, 3), activation='relu'),
        # MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation=None),
        Dense(num_classes, activation='softmax'),
    ])
    return model
classifier = Sequential()

In [46]:
# # Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens
# import keras.preprocessing.image as kp

# train_datagen = kp.ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

# validation_datagen = kp.ImageDataGenerator(rescale = 1./255)
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the input directory
input_dir = 'dataset_personagens\\training_set'
test_dir = 'dataset_personagens\\test_set'
# Parameters
img_size = 200  # Example size for MNIST (28x28)
categories = ['bart', 'homer']  # Replace with your actual categories

# Function to load and preprocess images
def load_images(input_dir, img_size, categories):
    images = []
    labels = []
    
    for category in categories:
        category_path = os.path.join(input_dir, category)
        class_num = categories.index(category)
        
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                # Load image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                # Resize image
                img = cv2.resize(img, (img_size, img_size))
                # Normalize image
                img = img / 255.0
                # Append to list
                images.append(img)
                labels.append(class_num)
    
    return np.array(images), np.array(labels)

# Load and preprocess images
images, labels = load_images(input_dir, img_size, categories)

test, tlabels =  load_images(test_dir, img_size, categories)
# Reshape images for CNN
images = images.reshape(-1, img_size, img_size, 1)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print("Data preprocessed successfully.")


Data preprocessed successfully.


In [47]:
X_train

array([[[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]],


       [[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
 

In [48]:
y_train

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0])

In [49]:
labels
input_shape = (img_size, img_size, 1)
model = build_cnn(input_shape, 2)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [50]:
# # Pré-processamento das imagens de treino e validação
# training_set = train_datagen.flow_from_directory('dataset_treino',
#                                                  target_size = (64, 64),
#                                                  batch_size = 32,
#                                                  class_mode = 'binary')

# validation_set = validation_datagen.flow_from_directory('dataset_validation',
#                                                         target_size = (64, 64),
#                                                         batch_size = 32,
#                                                         class_mode = 'binary')

In [51]:
# Executando o treinamento (esse processo pode levar bastante tempo, dependendo do seu computador)
#model.fit(X_train, y_train, steps_per_epoch = 8000, epochs = 5, validation_data = (X_val,y_val), validation_steps = 2000) Acc=72.6%
#Redução para 2000 passos por epoca
model.fit(X_train, y_train, steps_per_epoch = 2000, epochs = 5, validation_data = (X_val,y_val), validation_steps = 2000) #Acc = 76.71%

Epoch 1/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.5635 - loss: 5.6196 - val_accuracy: 0.6585 - val_loss: 0.6425
Epoch 2/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8832 - loss: 0.3339 - val_accuracy: 0.7317 - val_loss: 0.6113
Epoch 3/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9821 - loss: 0.0458 - val_accuracy: 0.6585 - val_loss: 0.9068
Epoch 4/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9941 - loss: 0.0113 - val_accuracy: 0.7073 - val_loss: 1.0408
Epoch 5/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9941 - loss: 0.0020 - val_accuracy: 0.6585 - val_loss: 1.0339


In [52]:
predictions = model.predict(test)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_labels == tlabels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Test Accuracy: 79.45%
